In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
%matplotlib qt5

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import sys

In [5]:
sys.path.insert(0, "/home/pleroy/DEV/processing/PoSAR-MC")
from posarmctools.readdata import *

In [6]:
sys.path.insert(0, "/home/pleroy/DEV/processing/focalization_python")
from posarutils.other.read_data_and_build_rd import *
from posarutils.other.PosarMCParameters import *

In [7]:
data_date = "2019_07_12_14_18_28"
data_dir = f"/home/pleroy/DATA/PoSAR-X/PIMA-1/2019_07_12/{data_date}"

In [8]:
params_filename = data_dir + "/" + data_date + "_parameters.xml"
params = PosarXParameters( params_filename )
params.print()

rampsPerBuffer 500
bufferSize 20000000.0
buffersPerFile 2
rampsPerFile 1000
fileSize 40000000.0
samplingFrequency 20000000.0
samplesPerRamp 20000
skipNSamples 0
rampPeriod 1000
startFrequency 9800
stopFrequency 10100
kc 416.785


In [9]:
buffersPerFile = params.buffersPerFile
samplesPerRamp = params.samplesPerRamp
rampsPerFile = params.rampsPerFile
samplesPerFile = samplesPerRamp * rampsPerFile

print( "buffersPerFile = {}, samplesPerRamp = {}, rampsPerFile = {}".format(
    buffersPerFile, samplesPerRamp, rampsPerFile )
     )

# selection in record
firstFile = 0
nbFiles = 47
lastFile = firstFile + nbFiles - 1
firstBuffer = firstFile * buffersPerFile

buffersPerFile = 2, samplesPerRamp = 20000, rampsPerFile = 1000


# Read the data

In [10]:
scalingFactor = 2 / 65535 
offset = -32768

In [11]:
timeSerie_A = np.zeros( samplesPerFile )
adc_A = np.zeros( (nbFiles, samplesPerFile) )

In [12]:
lastBuffer = (nbFiles-1) * buffersPerFile + firstBuffer
for k in range( nbFiles ):
    
    # read the data
    nb = str( int( k * buffersPerFile + firstBuffer) )
    filename = data_dir + "/record" + nb + ".bin"
    readFileADLINKCh0( filename, samplesPerFile, timeSerie_A )
    
    print("k = {}, block = {} / {}".format(k, nb, lastBuffer))
    
    adc_A[ k, : ] = (timeSerie_A + offset) * scalingFactor

k = 0, block = 0 / 92
k = 1, block = 2 / 92
k = 2, block = 4 / 92
k = 3, block = 6 / 92
k = 4, block = 8 / 92
k = 5, block = 10 / 92
k = 6, block = 12 / 92
k = 7, block = 14 / 92
k = 8, block = 16 / 92
k = 9, block = 18 / 92
k = 10, block = 20 / 92
k = 11, block = 22 / 92
k = 12, block = 24 / 92
k = 13, block = 26 / 92
k = 14, block = 28 / 92
k = 15, block = 30 / 92
k = 16, block = 32 / 92
k = 17, block = 34 / 92
k = 18, block = 36 / 92
k = 19, block = 38 / 92
k = 20, block = 40 / 92
k = 21, block = 42 / 92
k = 22, block = 44 / 92
k = 23, block = 46 / 92
k = 24, block = 48 / 92
k = 25, block = 50 / 92
k = 26, block = 52 / 92
k = 27, block = 54 / 92
k = 28, block = 56 / 92
k = 29, block = 58 / 92
k = 30, block = 60 / 92
k = 31, block = 62 / 92
k = 32, block = 64 / 92
k = 33, block = 66 / 92
k = 34, block = 68 / 92
k = 35, block = 70 / 92
k = 36, block = 72 / 92
k = 37, block = 74 / 92
k = 38, block = 76 / 92
k = 39, block = 78 / 92
k = 40, block = 80 / 92
k = 41, block = 82 / 92
k = 42,

In [14]:
A_reshaped = adc_A.reshape(nbFiles * rampsPerFile, samplesPerRamp)

## Spectrum

### No window

In [15]:
samplesPerDownRamp = int(samplesPerRamp/2)

In [16]:
fft_A = np.fft.ifft(A_reshaped[:, 0:samplesPerDownRamp], axis=1)

In [17]:
fft_abs = 20 * np.log10( np.abs( fft_A ) )

/home/pleroy/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log10
  """Entry point for launching an IPython kernel.


In [24]:
fig, ax = plt.subplots(1,1)
ax.imshow( fft_abs[::100, 500:2000] )
title = "no window " + data_date
ax.set_title(title)
fig.savefig( f"{data_dir}/{title}.png" )

### Hanning

In [19]:
hanning = np.hanning(samplesPerDownRamp)

In [20]:
fft_A_hanning = np.fft.ifft(A_reshaped[:, 0:samplesPerDownRamp]*hanning, axis=1)

In [21]:
fft_abs_hanning = 20 * np.log10( np.abs( fft_A_hanning ) )

In [22]:
fig, ax = plt.subplots(1,1)
ax.imshow( fft_abs_hanning[::100, 500:2000])
title = "hanning " + data_date
ax.set_title(title)
fig.savefig( f"{data_dir}/{title}.png" )

### Compare data

In [23]:
idxLine = 0
fig, ax = plt.subplots(1,1)
ax.plot( fft_abs[idxLine,:], label="raw"  )
ax.plot( fft_abs_hanning[idxLine,:], label="hanning"  )
ax.legend()
ax.grid()

# Build RD

**Check that the first ramp is an up ramp**

In [25]:
fig, ax = plt.subplots(1,1)
ax.plot( A_reshaped[0, :] )
title = "first acquisition " + data_date
ax.set_title(title)
ax.grid()
fig.savefig( f"{data_dir}/{title}.png", bbox_inches='tight')

**Set the shifted flag properly**

In [26]:
upOrDown = "Up"
#upOrDown = "Down"

if upOrDown == "Down":
    rampDownFirst = 1
    withHanning = 1
    RDc = build_rd_from_data_rampDown( params, A_reshaped, rampDownFirst, withHanning )
elif upOrDown == "Up":
    rampUpFirst = 0
    withHanning = 1
    RDc = build_rd_from_data_rampUp( params, A_reshaped, rampUpFirst, withHanning )

build rd from ramp up
shape of the samples matrix = (47000, 20000)
ramp down first in the data files
with Hanning window


## Save data

In [27]:
if withHanning:
    hanning = "_hanning"
else:
    hanning = ""

In [28]:
coupling_RD = np.average(RDc, 0)
coupling_name = f'{data_dir}/coupling_RD_files_{firstFile}_{lastFile}_ramp{upOrDown}{hanning}.npy'
print( f"load {coupling_name}")
np.save( coupling_name, coupling_RD )

load /home/pleroy/DATA/PoSAR-X/PIMA-1/2019_07_12/2019_07_12_14_18_28/coupling_RD_files_0_46_rampUp_hanning.npy


In [29]:
plt.figure()
plt.plot(np.abs(coupling_RD), 'k', label='coupling')
plt.legend()
plt.grid()

In [30]:
RD1_name = f'{data_dir}/RD_files_{firstFile}_{lastFile}_ramp{upOrDown}{hanning}.npy'
print( f"load {RD1_name}")
np.save( RD1_name, RDc) 

load /home/pleroy/DATA/PoSAR-X/PIMA-1/2019_07_12/2019_07_12_14_18_28/RD_files_0_46_rampUp_hanning.npy
